In [3]:
!pip install yfinance
!pip install investpy

In [1]:
import yfinance as yf
from investpy import stocks,crypto, get_crypto_historical_data
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import logging

import warnings

#Logging por padrão só exibe por padrão warning e critical, com o level configuro o nivel de exibição
logging.basicConfig(level=logging.INFO, filename="bronze.log", format="%(asctime)s - %(levelname)s - %(message)s")


In [2]:
# Coletar os dados de cada ticker
# Puxar todos os tickers de ações brasileiras
def listar_acoes_b3():
    """ 
    Lista todas as ações listadas na bolsa de valores B3
    """
    return stocks.get_stocks(country='brazil')    


acoes_b3 = listar_acoes_b3()

"""Normaliza as ações com o .SA no final"""
acoes_b3['acao'] = acoes_b3['symbol'].apply(lambda x: x + '.SA')

# Exibir as primeiras linhas da lista de ações
logging.info(acoes_b3)



In [3]:
from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
print(datetime.today())
print(datetime.today()-relativedelta(years=3))
data_inicial = (datetime.today()-relativedelta(years=3)).strftime('%d/%m/%Y')
data_final = (datetime.today()).strftime('%d/%m/%Y')

print(data_inicial)
print(data_final)



2024-11-27 20:41:32.518586
2021-11-27 20:41:32.518586
27/11/2021
27/11/2024


In [7]:
def fetch_stock_data(ticker, period="2y", interval="1d"):
    try:
        stock = yf.Ticker(ticker)
        data = stock.history(period=period)
        
        # Verifica se o DataFrame contém dados
        if data.empty:
            logging.warning(f"Ação {ticker} pode estar inativa ou excluída. Sem dados de preço.")
            return None
        else:
            return data
    except Exception as e:
        logging.error
        (f"Erro ao buscar dados para {ticker}: {e}")
        return None

In [9]:
lista_acoes = acoes_b3["acao"].tolist()


for lista_acao in lista_acoes:
    stock_data = yf.Ticker(lista_acao)
    print(stock_data)
    data = stock_data.history()  # Obtém dados diários dos últimos 2 anos

# Obtenha os dados históricos da ação


yfinance.Ticker object <ABCB4.SA>
yfinance.Ticker object <AGRO3.SA>
yfinance.Ticker object <RAIL3.SA>
yfinance.Ticker object <ALPA3.SA>
yfinance.Ticker object <ALPA4.SA>
yfinance.Ticker object <ALSO3.SA>
yfinance.Ticker object <AMAR3.SA>
yfinance.Ticker object <ABEV3.SA>
yfinance.Ticker object <ADHM3.SA>
yfinance.Ticker object <ARZZ3.SA>
yfinance.Ticker object <BBAS3.SA>
yfinance.Ticker object <BBDC3.SA>
yfinance.Ticker object <BBDC4.SA>
yfinance.Ticker object <BBRK3.SA>
yfinance.Ticker object <BEEF3.SA>
yfinance.Ticker object <BPHA3.SA>
yfinance.Ticker object <BPAN4.SA>
yfinance.Ticker object <BRAP3.SA>
yfinance.Ticker object <BRAP4.SA>
yfinance.Ticker object <BRFS3.SA>
yfinance.Ticker object <APER3.SA>
yfinance.Ticker object <BRKM3.SA>
yfinance.Ticker object <BRKM5.SA>
yfinance.Ticker object <BRML3.SA>
yfinance.Ticker object <BRPR3.SA>
yfinance.Ticker object <BRSR6.SA>
yfinance.Ticker object <OIBR3.SA>
yfinance.Ticker object <OIBR4.SA>
yfinance.Ticker object <BTOW3.SA>
yfinance.Ticke

KeyboardInterrupt: 

In [40]:
import yfinance as yf
import warnings
import pandas as pd

# Suprimir warnings se necessário
warnings.filterwarnings("ignore")

# Função para buscar dados históricos de uma ação
def fetch_stock_data(ticker, period="2y", interval="1d"):
    try:
        stock = yf.Ticker(ticker)
        data = stock.history(period=period, interval=interval)
        
        # Verifica se o DataFrame contém dados
        if data.empty:
            print(f"Ação {ticker} pode estar inativa ou excluída. Sem dados de preço.")
            return None
        else:
            data['Ticker'] = ticker  # Adiciona uma coluna para identificar a ação
            return data
    except Exception as e:
        print(f"Erro ao buscar dados para {ticker}: {e}")
        return None

# Lista de ações a serem consultadas
lista_acoes = acoes_b3["acao"].tolist()  # ou lista_acoes = cryptos["acao"].tolist()
# lista_acoes = ["PETR4.SA"]  # Exemplo de uso

# DataFrame final para consolidar os dados de ações ativas
dados_acoes = pd.DataFrame()
# DataFrame final para consolidar os dados de ações inativas
dados_acoes_inativas = pd.DataFrame()

# Itera sobre cada ação na lista e captura os dados válidos
for lista_acao in lista_acoes:
    data = fetch_stock_data(lista_acao)
    if data is not None:
        dados_acoes = pd.concat([dados_acoes, data])  # Adiciona dados ao DataFrame final
    if data is None:
        dados_acoes_inativas = pd.concat([dados_acoes_inativas, data])  # Adiciona dados ao DataFrame final

# Reseta o índice do DataFrame consolidado para melhor visualização
dados_acoes.reset_index(drop=True, inplace=True)

# Reseta o índice do DataFrame consolidado para melhor visualização
dados_acoes_inativas.reset_index(drop=True, inplace=True)

# Exibe o DataFrame consolidado ou salva em arquivo, conforme necessidade
print(dados_acoes.head())  # Exibe as primeiras linhas dos dados capturados
# Salva o DataFrame consolidado em um arquivo CSV, se desejado
dados_acoes.to_csv("dados_acoes_ativas.csv", index=False)

# Salva o DataFrame consolidado em um arquivo CSV -> Motivo: Vou usar para cruzaar com a investpy 
# e retirar estas ações da lista, evitando assim warnings desnecessários.
dados_acoes.to_csv("dados_acoes_inativas.csv", index=False)

$ALSO3.SA: possibly delisted; no price data found  (period=2y) (Yahoo error = "No data found, symbol may be delisted")


Ação ALSO3.SA pode estar inativa ou excluída. Sem dados de preço.


$ARZZ3.SA: possibly delisted; no price data found  (period=2y) (Yahoo error = "No data found, symbol may be delisted")


Ação ARZZ3.SA pode estar inativa ou excluída. Sem dados de preço.


$BBRK3.SA: possibly delisted; no price data found  (period=2y) (Yahoo error = "No data found, symbol may be delisted")


...

Ação BMGB11.SA pode estar inativa ou excluída. Sem dados de preço.


$SDIP11.SA: possibly delisted; no price data found  (period=2y) (Yahoo error = "No data found, symbol may be delisted")


Ação SDIP11.SA pode estar inativa ou excluída. Sem dados de preço.


$TCPF11.SA: possibly delisted; no price data found  (period=2y) (Yahoo error = "No data found, symbol may be delisted")


Ação TCPF11.SA pode estar inativa ou excluída. Sem dados de preço.
        Open       High        Low      Close   Volume  Dividends  \
0  18.998293  18.998293  18.438759  18.438759   716674        0.0   
1  18.318243  18.361284  17.620980  17.775927  1660144        0.0   
2  17.775927  18.120254  17.431599  18.034172  1090862        0.0   
3  17.896443  18.034174  17.354127  17.491858  1145904        0.0   
4  17.095880  17.844794  17.070055  17.784536   946834        0.0   

   Stock Splits    Ticker  Capital Gains  
0           0.0  ABCB4.SA            NaN  
1           0.0  ABCB4.SA            NaN  
2           0.0  ABCB4.SA            NaN  
3           0.0  ABCB4.SA            NaN  
4           0.0  ABCB4.SA            NaN  
